# Results

Testing networks and saving results

TODO:
- clean up repeated bits into functions
- save csv files with results to share/use elsewhere

In [1]:
# imports
import os
import yaml
from functools import reduce
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
from architectures import *
from scipy.stats import bayes_mvs, entropy, linregress, spearmanr

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor
bmvs = bayes_mvs

## Data Loading

In [2]:
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/temp')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')

data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))
df = hvhdata[0]
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])

defmod = np.loadtxt(os.path.expanduser('~/Downloads/loglik_hvh_final.txt')).reshape([40, 5])

## Compile Results

In [3]:
def unfreeze(net):
    for layer in L.get_all_layers(net.net):
        for param in layer.params:
            layer.params[param].add('trainable')
    net.params = L.get_all_params(net.net, trainable=True)
    
    return None

def load_arch_specs(archname):
    with open('arch_specs.yaml') as archfile:
        arch = yaml.load(archfile)
    return arch[archname]

def load_pretrain_results(archname, archfunc, idx, net=None):
    specs = load_arch_specs(archname)
    arch = lambda input_var: archfunc(input_var, **specs['kwargs'])
    fname = '{} {} split agg fit exp 1-4.npz'.format(archname, idx)
    resdf = pd.DataFrame(index=np.arange(Xt.shape[0]), columns=[idx])

    if net is None:
        net = Network(arch)
    
    net.load_params(os.path.join(paramsdir, fname))
    res = net.itemized_test_fn(Xt, yt)
    resdf[idx] = res
    return resdf, net

def load_train_results(archname, archfunc, idx, test_idx, net=None):
    specs = load_arch_specs(archname)
    arch = lambda input_var: archfunc(input_var, **specs['kwargs'])
    fname = '{} {} agg fit exp 1-4 {} tune fit exp 0.npz'.format(archname, idx, test_idx)
    resdf = pd.DataFrame(index=np.arange(Xt.shape[0]), columns=[idx])
    
    if net is None:
        net = Network(arch)
    
    net.load_params(os.path.join(paramsdir, fname))
    
    group_idx = (test_idx - 1) % 5
    selection = df.loc[df['group']==(group_idx+1)].index.values
    res = net.itemized_test_fn(Xt[selection, :, :, :], yt[selection])
    resdf.loc[selection, idx] = res
    return resdf, net

In [7]:
archname = 'h4'
archfunc = multiconvX_ws
Xt, yt, _, _, _ = loading.unpack_data(df)

pretrain_nets = []
pretrain_results = []
# net = Network()

for idx in range(5):
    resdf, net = load_pretrain_results(archname, archfunc, idx)
    pretrain_nets.append(net)
    pretrain_results.append(resdf)

pretrain_results = pd.concat(pretrain_results, axis=1)
pretrain_results.to_csv(os.path.join(resultsdir, 'pretrain {}.csv'.format(archname)))

In [8]:
pretrain_results['mean'] = pretrain_results.mean(axis=1)
pretrain_results.pivot_table(index=df['subject'], values='mean').mean()

1.982299856690378

In [9]:
train_nets = []
train_results = []
for idx in range(5):
    for test_idx in range(5):
        print(idx, test_idx)
        resdf, net = load_train_results(archname, archfunc, idx, test_idx)
        train_nets.append(net)
        train_results.append(resdf)

train_results = pd.concat(train_results, axis=1, join='inner').stack().unstack()
train_results.to_csv(os.path.join(resultsdir, 'train {}.csv'.format(archname)))

0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4


In [10]:
train_results['mean'] = train_results.mean(axis=1)
bmvs(train_results.pivot_table(index=df['subject'], values='mean').values)

(Mean(statistic=1.9549269570194059, minmax=(1.8640069212399626, 2.045846992798849)),
 Variance(statistic=0.12277406515959045, minmax=(0.083240882724647969, 0.17678814527682843)),
 Std_dev(statistic=0.34803206050705632, minmax=(0.28851496100661395, 0.42046182380428837)))

# Response Times

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
plt.rc('text', usetex=True)

%matplotlib inline

In [ ]:
scatterkws = {
    'linestyle': 'none', 
    'marker': 'o', 'markerfacecolor': (.2, .2, .2), 'markeredgecolor': 'black', 
    'alpha': .3
}

histkws = {
    'edgecolor': 'white'
}

In [ ]:
def hicks_entropy(pred):
    H = pred * np.log2(1 / (pred + 1))
    return H.sum(axis=1)

In [ ]:
X, y, S, G, Np = loading.unpack_data(df)
df['mean corrected rt'] = 0
for subject in df['subject'].unique():
    fil = df['subject'] == subject
    df.loc[fil, 'mean corrected rt'] = df.loc[fil, 'rt'] - df.loc[fil, 'rt'].mean()

rt = df['mean corrected rt']

In [ ]:
# compute mean entropy for each test group
E = []
for split_idx in range(25):
    N = train_nets[split_idx]
    locs = np.where(G==(split_idx//5))[0]
    L = N.output_fn(X[locs, :, :, :])
    E.append(hicks_entropy(L))

for g in range(5):
    df.loc[df['group']==(g+1), 'entropy'] = np.array(E[g*5:(g+1)*5]).T.mean(axis=1)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10, 6))

x = df['entropy']
y = np.log(df['rt']/1000)
axes.plot(x, y, **scatterkws)
lr = linregress(x, y)
pval = lr.pvalue if lr.pvalue >= .001 else .001
axes.text(.05, .05, r"r = {:.2f}, p $<$ {:.3f}".format(lr.rvalue, pval), transform=axes.transAxes, fontsize=14)
plt.setp(axes, xlabel=r"$\textrm{Entropy}$", ylabel=r'$\log{\textrm{Response time (s)}}$', ylim=[-5, 5])

sns.despine()

Hick's law holds (ish).

# Values

In [ ]:
gendata = pd.read_csv(
    os.path.join(headdir, 'Data/1_gen/Clean/_summaries/all_evals_model_input.csv'),
    names=['subject', 'color', 'bp', 'wp', 'zet', 'rt', 'val']
)
gendata['group'] = -1

X, y, S, G, Np = loading.unpack_data(gendata)

In [ ]:
N = train_nets[0]
logistic = lambda x: 1 / (1 + np.exp(-x))
zscore = lambda x: (x - x.mean()) / (x.std() / np.sqrt(x.shape[0]))
Vr = N.value_fn(X)
V = Vr.sum(axis=1)
Vl = 7*logistic(zscore(V))

V2 = np.zeros_like(V)
yz = np.zeros_like(y)
for subject in range(S.max()):
    V2[S==subject] = zscore(V[S==subject])
    yz[S==subject] = zscore(y[S==subject])
    
V2l = 7*logistic(V2)

In [ ]:
Vr = N.value_fn(X) - N.value_fn(X[:, ::-1, :, :])

In [ ]:
plt.hist(V, **histkws)
sns.despine()

In [ ]:
plt.hist(V2, **histkws) #, bins=np.arange(0, 8, .5), **histkws)
sns.despine()

In [ ]:
plt.plot(V, gendata['val'], **scatterkws)
print(linregress(V2, gendata['val']))
sns.despine()

In [ ]:
plt.plot(gendata['val'], gendata['zet'], **scatterkws)
print(linregress(gendata['zet'], gendata['val']))
sns.despine()

In [ ]:
plt.plot(zscore(V), yz, **scatterkws)
print(linregress(zscore(V), yz))
sns.despine()

In [ ]:
gendata['valhat'] = 6*logistic(V2) + 1
gendata['valhat'] = gendata['valhat'].map(int)
gendata['position'] = gendata['bp'] + gendata['wp']
gp = gendata.pivot_table(index='position', columns='zet', values='group', aggfunc=len, fill_value=0)
gvp = gendata.pivot_table(index='position', values='valhat')
gp['valhat'] = gvp.values
gp['valsum'] = gp[list(np.arange(1, 8, 1))].values.argmax(axis=1) + 1
gp.head()

In [ ]:
linregress(gp['valhat'], gp['valsum'])